In [3]:
import scanpy as sc
import scprep
import phate
import numpy as np
import SPARC
import gspa

### Preprocess

In [32]:
adata = sc.datasets.visium_sge(sample_id="V1_Human_Lymph_Node")
adata.var_names_make_unique()
adata.var["mt"] = adata.var_names.str.startswith("MT-")
sc.pp.calculate_qc_metrics(adata, qc_vars=["mt"], inplace=True)
sc.pp.filter_cells(adata, min_counts=5000)
sc.pp.filter_cells(adata, max_counts=35000)
adata = adata[adata.obs["pct_counts_mt"] < 20]
print(f"#cells after MT filter: {adata.n_obs}")
sc.pp.filter_genes(adata, min_cells=10)
sc.pp.normalize_total(adata, inplace=True)
sc.pp.log1p(adata)
sc.pp.highly_variable_genes(adata, flavor="seurat", n_top_genes=2000)

/home/aarthivenkat/.local/lib/python3.8/site-packages/anndata/_core/anndata.py:1830: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


#cells after MT filter: 3861


/home/aarthivenkat/.local/lib/python3.8/site-packages/scanpy/preprocessing/_simple.py:251: ImplicitModificationWarning: Trying to modify attribute `.var` of view, initializing view as actual.
  adata.var['n_cells'] = number


In [33]:
adata.write('data/V1_Human_Lymph_Node/processed.h5ad')

### Get SPARC operator

In [4]:
adata = sc.read_h5ad('data/V1_Human_Lymph_Node/processed.h5ad')

In [5]:
sparc_op = SPARC.spARC(n_jobs=-1, random_state=42)
data_sparc = sparc_op.fit_transform(expression_X = adata.to_df(),
                                    spatial_X = adata.obs[['array_row', 'array_col']])

Calculating spARC...
  Calculating PCA...
  Calculated PCA in 3.20 seconds.
  Calculating expression graph...
  Calculated expression graph in 0.99 seconds.
  Calculating spatial graph...
  Calculated spatial graph in 1.48 seconds.
  Calculating random walks on expression graph...
  Calculated random walks on expression graph in 0.39 seconds.
  Calculating random walks on spatial graph...
  Calculating spARCed expression data...
  Calculated spARCed expression data in 1.44 seconds.
Calculated spARC in 7.51 seconds.


In [6]:
integrated_diff_op = sparc_op.expression_diff_op @ sparc_op.spatial_diff_op

## GSPA

In [7]:
# GSPA operator constructs wavelet dictionary with integrated diffusion operator
# When inputting diffusion operator, GSPA operator does not need to construct graph or diffusion operator
gspa_op = gspa.GSPA(diffusion_operator = integrated_diff_op)
gspa_op.build_wavelet_dictionary()

100%|██████████| 6/6 [00:18<00:00,  3.07s/it]


In [10]:
data_hvg, hvgs = scprep.select.highly_variable_genes(adata.to_df(), adata.var_names, percentile=90)

In [11]:
gene_signals = data_hvg.T # embed all highly variable genes
gene_ae, gene_pc = gspa_op.get_gene_embeddings(gene_signals)
gene_localization = gspa_op.calculate_localization()

Epoch 1/100
59/59 [==============================] - 0s 4ms/step - loss: 0.0034 - val_loss: 0.0029
Epoch 2/100
59/59 [==============================] - 0s 3ms/step - loss: 0.0026 - val_loss: 0.0024
Epoch 3/100
59/59 [==============================] - 0s 3ms/step - loss: 0.0023 - val_loss: 0.0022
Epoch 4/100
59/59 [==============================] - 0s 3ms/step - loss: 0.0021 - val_loss: 0.0021
Epoch 5/100
59/59 [==============================] - 0s 3ms/step - loss: 0.0020 - val_loss: 0.0020
Epoch 6/100
59/59 [==============================] - 0s 3ms/step - loss: 0.0019 - val_loss: 0.0020
Epoch 7/100
59/59 [==============================] - 0s 3ms/step - loss: 0.0019 - val_loss: 0.0019
Epoch 8/100
59/59 [==============================] - 0s 3ms/step - loss: 0.0018 - val_loss: 0.0019
Epoch 9/100
59/59 [==============================] - 0s 3ms/step - loss: 0.0018 - val_loss: 0.0019
Epoch 10/100
59/59 [==============================] - 0s 3ms/step - loss: 0.0018 - val_loss: 0.0019
Epoch 11/

In [12]:
np.savez('./results/GSPA_QR.npz', signal_embedding=gene_ae,
         localization_score=gene_localization, genes=hvgs)